In [ ]:

from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain_openai import ChatOpenAI

# from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import json
from random import seed,shuffle
from time import sleep

In [ ]:
from ragchat.html_cleaner import HtmlCleaner
from ragchat.html_patterns import HtmlPatterns


In [ ]:
cleaner = HtmlCleaner(parser="inscriptis", debug=True, max_pages=None)
clean_text_dict = cleaner.get_clean_text_dict()


In [ ]:
l_lens = []
c_lens = []
for fname in list(clean_text_dict.keys()):
    cleaned = clean_text_dict[fname]["cleaned"]
    c_lens.append((len(cleaned), fname))
    l_lens.append((len(cleaned.splitlines()), fname))

c_lens.sort()
l_lens.sort()


In [ ]:
idx = -1
pg = c_lens[idx][1]
text = clean_text_dict[pg]["cleaned"]
title = clean_text_dict[pg]["title"]
all_text = clean_text_dict[pg]["all_text"]


In [ ]:
print(title, text)


In [ ]:
doc = Document(
    page_content=text,
    metadata={"page": pg, "title": title},
)


In [ ]:
document_prompt = PromptTemplate.from_template("title: {title} \n\n\n{page_content}")
partial_format_document = partial(format_document, prompt=document_prompt)


In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)


In [ ]:
map_chain = (
    {"context": partial_format_document}
    | PromptTemplate.from_template("Summarize this content:\n\n{context}")
    # | PromptTemplate.from_template("Create a detailed summary of this:\n\n\n\n{context}"
    | llm
    | StrOutputParser()
)


In [ ]:
map_chain.invoke(doc)


In [ ]:
map_chain2 = (
    {"context": partial_format_document}
    | PromptTemplate.from_template(
        "Create an extremely detailed summary of this:\n\n\n\n{context}"
    )
    | llm
    | StrOutputParser()
)


In [ ]:
map_chain2.invoke(doc)  # excludes "content"


In [ ]:
llm.get_num_tokens(text)
